In [1]:
# Import modules

import pandas as pd
import numpy as np
import sys
import nltk
import pickle

import torch
import torch.nn as nn
import torch.utils.data as utils
import torchwordemb
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms

from sklearn.metrics import log_loss

from multiprocessing import Pool

sys.path.append('/home/hugoperrin/Bureau/Data science/Kaggle/ToxicComment/Models/')
from CNN_1d import CNN

sys.path.append('/home/hugoperrin/Bureau/Data science/Kaggle/ToxicComment/Models/')
from utils import train, predict

%matplotlib inline

In [2]:
# Import data
train_vect = np.load('/home/hugoperrin/Bureau/Datasets/ToxicComment/Comment2Vec_train.npy')
# test_vect = np.load('/home/hugoperrin/Bureau/Datasets/ToxicComment/Comment2Vec_test.npy')

Xtrain = pd.read_csv('/home/hugoperrin/Bureau/Datasets/ToxicComment/train.csv')
list_classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
targets = Xtrain[list_classes].values

# Xtest = pd.read_csv('/home/hugoperrin/Bureau/Datasets/ToxicComment/test.csv')
# final_id = Xtest['id']

del Xtrain #, Xtest

In [3]:
# Preprocess data for torch
train_vect = train_vect.reshape(train_vect.shape[0],1,train_vect.shape[1])
# test_comments = test_vect.reshape(test_vect.shape[0],1,test_vect.shape[1])
# del train_vect, test_vect

# train_comments.shape, test_comments.shape

In [4]:
# # Train test split
train_comments = train_vect[:130000,:,:]
valid_comments = train_vect[130001:145000,:,:]
test_comments = train_vect[145001:]

train_labels = targets[:130000,:]
valid_labels = targets[130001:145000,:]
test_labels = targets[145001:,:]

del targets, train_vect

In [5]:
%%time

predictions = pd.DataFrame(index=range(len(test_comments)))

for target in list_classes:
    
    print("Estimation of {}".format(target))

    labels_train = train_labels[:,list_classes.index(target)]
    labels_train = labels_train.reshape(labels_train.shape[0],1)

    labels_valid = valid_labels[:,list_classes.index(target)]
    labels_valid = labels_valid.reshape(labels_valid.shape[0],1)

    labels_test = test_labels[:,list_classes.index(target)]
    labels_test = labels_test.reshape(test_labels.shape[0],1)

    use_GPU = True

    batch_size = 512
    num_epoch = 5

    train_dataset = torch.utils.data.TensorDataset(torch.FloatTensor(train_comments), 
                                                   torch.FloatTensor(labels_train))

    valid_dataset = torch.utils.data.TensorDataset(torch.FloatTensor(valid_comments), 
                                                   torch.FloatTensor(labels_valid))

    test_dataset = torch.FloatTensor(test_comments)


    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                               batch_size=batch_size, 
                                               shuffle=True, 
                                               num_workers = 8)

    valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                               batch_size=batch_size,
                                               shuffle=False, 
                                               num_workers = 8)

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                               batch_size=batch_size,
                                               shuffle=False, 
                                               num_workers = 8)

    net = CNN()
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.RMSprop(net.parameters(), lr=0.00002, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0.9)

    train(num_epoch, net, train_loader, optimizer, criterion, valid_loader=None, use_GPU=use_GPU)

    predictions[target] = predict(net, test_loader, use_GPU=use_GPU)

print('\n')


>> LEARNING: 112691 parameters

Epoch: 1, step:   100, training loss: 0.2810
Epoch: 1, step:   200, training loss: 0.1730
Epoch: 2, step:   300, training loss: 0.0698
Epoch: 2, step:   400, training loss: 0.1509
Epoch: 2, step:   500, training loss: 0.1504
Epoch: 3, step:   600, training loss: 0.1333
Epoch: 3, step:   700, training loss: 0.1444
Epoch: 4, step:   800, training loss: 0.0518
Epoch: 4, step:   900, training loss: 0.1418
Epoch: 4, step:  1000, training loss: 0.1412
Epoch: 5, step:  1100, training loss: 0.1192
Epoch: 5, step:  1200, training loss: 0.1394

>> LEARNING: 112691 parameters

Epoch: 1, step:   100, training loss: 0.1166
Epoch: 1, step:   200, training loss: 0.0373
Epoch: 2, step:   300, training loss: 0.0129
Epoch: 2, step:   400, training loss: 0.0290
Epoch: 2, step:   500, training loss: 0.0292
Epoch: 3, step:   600, training loss: 0.0248
Epoch: 3, step:   700, training loss: 0.0265
Epoch: 4, step:   800, training loss: 0.0099
Epoch: 4, step:   900, training lo

In [6]:
score = 0

for i in range(len(list_classes)):
    score += log_loss(test_labels[:,i],predictions.iloc[:,i])*(1/len(list_classes))
    
score

0.063295737412642966

In [7]:
# predictions.to_csv('/home/hugoperrin/Bureau/Datasets/ToxicComment/1st_submission.csv')